<a href="https://colab.research.google.com/github/Catisyf/Python-projects-chez-Toucan/blob/main/Backfill_text_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install -U sentence-transformers

In [ ]:
!ls '/content/drive/My Drive/crm_contacts.csv'

'/content/drive/My Drive/crm_contacts.csv'


In [4]:
import pandas as pd
from scipy import spatial
from collections import Counter #reduce processing time
import numpy as np

In [5]:
df_contacts = pd.read_csv('/content/drive/My Drive/crm_contacts.csv', na_values=' ') #replace blank cells with na
df_persona = pd.read_csv('/content/drive/My Drive/buyer_persona_definition.csv')

## **Data Preparation**

Step 1: remove null & irrelevant values from CRM data

Step 2: remove stop words in both datasets (of, de etc.). It is not necessary to use stopwords dictionaries.  

In [6]:
df_contacts = df_contacts.replace('?', 'np.Nan').dropna() #remove null

In [7]:
df_contacts = df_contacts[~df_contacts.jobtitle.str.contains('(site)|(try-for-free)')] #remove strings that contain irrelevant info

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [8]:
stopwords = ['of', 'de', 'du', 'for', 'For', 'the', 'et', '&', '/'] 
space = ' '
stopwords_dict = Counter(stopwords)

In [9]:
clean = []
for n in range(0,len(df_contacts)):
  clean.append(space.join([word for word in df_contacts.iloc[n]['jobtitle'].split() \
                                                if word not in stopwords_dict])) #remove stopwords from persona data
  
df_contacts['clean_title'] = clean

In [10]:
clean = []
for n in range(0,len(df_persona)):
  clean.append(space.join([word for word in df_persona.iloc[n]['job_title'].split() \
                                                if word not in stopwords_dict])) #remove stopwords from persona data
  
df_persona['clean_title'] = clean

## **Use NLP model for matching**

In [11]:
#Pre-trained sentence transformer model
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [12]:
#Vectorize cleaned job titles

vector1 = []
for i in range(0,len(df_contacts)):
  vector1.append(model.encode(df_contacts.iloc[i]['clean_title']))

df_contacts['vector'] = vector1

In [24]:
vector2 = []
for i in range(0,len(df_persona)):
  vector2.append(model.encode(df_persona.iloc[i]['clean_title']))

df_persona['vector'] = vector2

In [ ]:
#function for cosine similarity
#def cosine_similarity(v1,v2):
  #return 1-spatial.distance.cosine(v1, v2)

In [34]:
vec_crm = []
vec_persona = []
similarity = []
for vec1 in df_contacts['vector']:
  for vec2 in df_persona['vector']:
    if (1-spatial.distance.cosine(vec1, vec2)) >= 0.85: #set thereshold for high-level similarity 
      vec_crm.append(vec1)
      vec_persona.append(vec2)
      similarity.append(1-spatial.distance.cosine(vec1, vec2))

In [59]:
table = pd.DataFrame({'vec_crm': vec_crm,
                     'vec_persona': vec_persona,
                      'similarity': similarity})

In [112]:
#find job titles & persona
table_persona = df_persona[df_persona['vector'].isin(vec_persona)]
table_contacts = df_contacts[df_contacts['vector'].isin(vec_crm)]

In [86]:
#convert np array to str so I can use the vector as a key for joining tables
list1 = []
list2 = []
for i in range(len(table)):
  list1.append(np.array_str(table.iloc[i]['vec_crm']))
  list2.append(np.array_str(table.iloc[i]['vec_persona']))

table['vec1'] = list1
table['vec2'] = list2

In [ ]:
ls = []
for i in range(len(table_contacts)):
  ls.append(np.array_str(table_contacts.iloc[i]['vector']))

table_contacts['vec_str'] = ls

In [ ]:
ls = []
for i in range(len(table_persona)):
  ls.append(np.array_str(table_persona.iloc[i]['vector']))

table_persona['vec_str'] = ls

In [97]:
#join table
table1 = table.merge(table_contacts, left_on = 'vec1', right_on = 'vec_str')

In [102]:
table2 = table1.merge(table_persona, left_on = 'vec2', right_on = 'vec_str')

In [106]:
final = table2[['similarity', 'jobtitle', 'email', 'buyer_persona', 'job_title']]

In [110]:
#final output
final.drop_duplicates()

,similarity,jobtitle,email,buyer_persona,job_title
0,0.874570,Senior Engineer,chibuzor.obilom@gmail.com,Practitioner,Senior BI Engineer
1,0.874570,Senior Engineer,vishal.dwivedi9@gmail.com,Practitioner,Senior BI Engineer
2,0.874570,Senior Engineer,skadir@tereos.com,Practitioner,Senior BI Engineer
9,0.879964,Senior Associate Engineer,rkalavakuntla@tasengg.com,Practitioner,Senior BI Engineer
10,0.932112,Manager of Sales,iamguan@yeah.net,Business Leader,Sales Director
...,...,...,...,...,...
2049,0.900356,Business Intelligence Project Leader,kasper.vrees@danone.com,Practitioner,Chef De Projet Business Intelligence
2050,0.850429,Chargée de projet Business Intelligence,claire.jourdan-sestier@isere.fr,Practitioner,Chef De Projet Business Intelligence
2051,0.866969,Business Intelligence Project Leader,kasper.vrees@danone.com,Practitioner,Chef de Projet Business Intelligence
2052,0.866108,Respo Analyse Données Finance,michel.ndiaye@fnacdarty.com,Practitioner,Financial Analyst
